In [6]:
! nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [8]:
!pip install cmake --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 49.3 MB/s eta 0:00:00
  Attempting uninstall: cmake
    Found existing installation: cmake 3.27.9
    Uninstalling cmake-3.27.9:
      Successfully uninstalled cmake-3.27.9


In [9]:
! wget ftp://ftp.gromacs.org/gromacs/gromacs-2023.3.tar.gz

--2023-12-18 21:32:41--  ftp://ftp.gromacs.org/gromacs/gromacs-2023.3.tar.gz
           => ‘gromacs-2023.3.tar.gz’
Resolving ftp.gromacs.org (ftp.gromacs.org)... 130.237.11.165, 2001:6b0:1:1191:216:3eff:fec7:6e30
Connecting to ftp.gromacs.org (ftp.gromacs.org)|130.237.11.165|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /gromacs ... done.
==> SIZE gromacs-2023.3.tar.gz ... 42071770
==> PASV ... done.    ==> RETR gromacs-2023.3.tar.gz ... done.
Length: 42071770 (40M) (unauthoritative)

gromacs-2023.3.tar. 100%[===================>]  40.12M  16.1MB/s    in 2.5s    

2023-12-18 21:32:46 (16.1 MB/s) - ‘gromacs-2023.3.tar.gz’ saved [42071770]



In [14]:

import os

if not os.path.isdir("/content/drive/MyDrive"):
  from google.colab import drive
  drive.mount("/content/drive")

if not os.path.isdir("/content/drive/MyDrive"):
  raise RuntimeError("Error: could not connect to Google Drive")

storage = "/content/drive/MyDrive/gromacs-on-colab"
os.makedirs(storage, exist_ok=True)
%env STORAGE={storage}

if "START" not in os.environ or not os.environ["START"]:
  %env START={os.getcwd()}
else:
  %cd {os.environ["START"]}

env: STORAGE=/content/drive/MyDrive/gromacs-on-colab
/content


In [ ]:
%%bash
# @markdown If not available, it is instead compiled from source code. (This takes a while.)

if [[ -d "/usr/local/gromacs" ]]; then
  exit 0  # already installed
fi

gromacs_vers="2023.3" #@param {type: "string"}
cache_gromacs="${STORAGE}/gromacs-${gromacs_vers}.tar.gz"

if [[ -s "${cache_gromacs}" ]]; then
  tar -xzf "${cache_gromacs}" -C "/usr/local"
  # Prebuilt archive not available, so download the source code and build it...
else
  wget "https://ftp.gromacs.org/gromacs/gromacs-2023.3.tar.gz"
  if [[ ! -s "gromacs-${gromacs_vers}.tar.gz" ]]; then
    echo "Error: could not download: gromacs-${gromacs_vers}.tar.gz" >&2
    exit 1
  fi
  tar -xzf "gromacs-${gromacs_vers}.tar.gz"
  rm "gromacs-${gromacs_vers}.tar.gz"

  cd "gromacs-${gromacs_vers}"
  mkdir "build"
  cd "build"
  cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA
  make -j $(nproc)
  make install # -> /usr/local/gromacs

    # Cache
  tar -czf "my_gromacs.tar.gz" -C "/usr/local" "gromacs"
  mv "my_gromacs.tar.gz" "${cache_gromacs}"
fi

In [21]:
import matplotlib
# Force matplotlib to not use any Xwindows backend.
matplotlib.use('Agg')

import glob
import random
import MDAnalysis

from scipy.spatial import Voronoi

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import LogNorm
from scipy.spatial import cKDTree
from tqdm import tqdm


def plot_frame_transitions(df, frames, title, lag_time=2, phi_offset=60, psi_offset=-90):
    phi_s, phi_e = -180 + phi_offset, 180 + phi_offset
    psi_s, psi_e = -180 + psi_offset, 180 + psi_offset

    # "improved" x/y axis ticks
    plt.xticks(np.arange(-180, 180, 30), np.arange(phi_s, phi_e, 30))
    plt.yticks(np.arange(-180, 180, 30), np.arange(psi_s, psi_e, 30))

    # axis labels (right order?)
    plt.xlabel('$\phi$')
    plt.ylabel('$\psi$')

    # 1:1 aspect ratio
    plt.axes().set_aspect('equal')
    # remove grid lines
    plt.axes().grid(False)

    if len(title):
        plt.title(title)

    sd = df.as_matrix()
    plt.scatter(sd[0], sd[1], color='red')

    for key in tqdm(frames.keys(), desc="Plotting transition paths"):
        prev = df.iloc[key]
        c = np.random.rand(3,)
        for step in range(lag_time):
            curr = df.iloc[key + step]
            plt.plot([prev[0], curr[0]], [prev[1], curr[1]], color=c, linewidth=1.0)
            prev = curr

    return plt


def plot_groups(df, c_centers, phi_offset=60, psi_offset=-90):
    phi_s, phi_e = -180 + phi_offset, 180 + phi_offset
    psi_s, psi_e = -180 + psi_offset, 180 + psi_offset

    tree = cKDTree(c_centers)
    points = np.c_[df['phi'], df['psi']]  # fancy zipping
    queries = tree.query(points)[1]
    plt.figure()
    plt.scatter(df['phi'], df['psi'], c=queries, s=5, linewidths=0.25)

    # "improved" x/y axis ticks
    plt.xlim([-180, 180])
    plt.ylim([-180, 180])
    # "improved" x/y axis ticks
    plt.xticks(np.arange(-180, 180, 30), np.arange(phi_s, phi_e, 30))
    plt.yticks(np.arange(-180, 180, 30), np.arange(psi_s, psi_e, 30))

    # axis labels (right order?)
    plt.ylabel('$\psi$')
    plt.xlabel('$\phi$')

    # 1:1 aspect ratio
    plt.axes().set_aspect('equal')
    # remove grid lines
    plt.axes().grid(False)

    return plt


def create_ramachandran_plot(df, title, colorbar=True, phi_offset=60, psi_offset=-90):
    phi_s, phi_e = -180 + phi_offset, 180 + phi_offset
    psi_s, psi_e = -180 + psi_offset, 180 + psi_offset

    plt.hist2d(df[0], df[1],
               range=[[-180, 180], [-180, 180]],  # not really necessary
               bins=360,
               # cmap='viridis',  # cf. http://matplotlib.org/examples/color/colormaps_reference.html
               norm=LogNorm())
    if colorbar:
        plt.colorbar()

    # "improved" x/y axis ticks
    plt.xticks(np.arange(-180, 180, 30), np.arange(phi_s, phi_e, 30))
    plt.yticks(np.arange(-180, 180, 30), np.arange(psi_s, psi_e, 30))

    # axis labels (right order?)
    plt.xlabel('$\phi$')
    plt.ylabel('$\psi$')

    # 1:1 aspect ratio
    plt.axes().set_aspect('equal')
    # remove grid lines
    plt.axes().grid(False)

    if len(title):
        plt.title(title)

    return plt


def plot_voronoi_ridges(ridges, linewidth=1):
    for segment in ridges:
        plt.plot([segment[0][0], segment[1][0]], [segment[0][1], segment[1][1]], linewidth=linewidth, color='black')


def plot_sampled_frames(frames, linewidth=1, n=-1):
    sample_frames = pd.DataFrame(frames)
    sd = sample_frames.as_matrix()
    plt.scatter(sd[0][:n], sd[1][:n], color='red', linewidth=linewidth)
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm


def save_frames_as_pdb(frames, u):
    # save all frames in pdb files
    for idx in tqdm(frames.keys(), desc="Saving GRO files for each microstate"):
        u.trajectory[idx]
        u.atoms.write('../data/frames/f' + str(idx) + '.gro')


def csv_print(mat):
    np.savetxt(sys.stdout, mat, fmt='%.5f', newline="\n")


def read_dihedral_angles(phi_path, psi_path, frames=1, phi_offset=60, psi_offset=-90):
    phi_df = pd.read_csv(phi_path, header=None,
                         index_col='frame', names=['frame', 'phi'],
                         delimiter='\t').head(frames)
    psi_df = pd.read_csv(psi_path, header=None,
                         index_col='frame', names=['frame', 'psi'],
                         delimiter='\t').head(frames)

    phi_df = (((phi_df + 180) + phi_offset) % 360) - 180
    psi_df = (((psi_df + 180) + psi_offset) % 360) - 180

    both_angles = pd.concat([phi_df['phi'], psi_df['psi']], axis=1, keys=['phi', 'psi']) * -1

    return pd.DataFrame(both_angles)


def read_xvg(fname):
    """Read columns of data from file fname

    Returns numpy array of data
    """
    skip = 0
    with open(fname, 'r') as f:
        for i, line in enumerate(f):
            if not line.startswith(('#', '@')):
                skip = i
                break

    return np.genfromtxt(fname, skip_header=skip, usecols=(0, 1))


def apply_offset(df, phi_offset=60, psi_offset=-90, phi_mult=1, psi_mult=1):
    phi = df[0]
    psi = df[1]

    phi = (((phi + 180) + phi_offset) % 360) - 180
    psi = (((psi + 180) + psi_offset) % 360) - 180

    phi *= phi_mult
    psi *= psi_mult

    return pd.DataFrame(pd.concat([phi, psi], axis=1))

import shlex
import subprocess
import datetime
import os
import pickle
import time
from sympy.geometry import *

import numpy as np
from scipy.spatial import cKDTree
from sklearn.cluster import MeanShift, estimate_bandwidth
from tqdm import tqdm


def clustering(df):
    # MeanShift clustering
    # The following bandwidth can be automatically detected using
    bandwidth = estimate_bandwidth(df.as_matrix(), quantile=0.2, n_samples=500)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(df.as_matrix())

    return ms.cluster_centers_


def sample_boundaries(df, ridges, dist=15, samples=10):
    # now sample frames and check if those points are near the voronoi ridges
    delta_dist = dist  # in degree
    frames = {}
    times = []
    with tqdm(total=samples, desc="sampling frames") as pbar:
        # sample transition 'positions'
        while len(frames) < samples:
            start = time.time()  # measure the time spend searching
            # choose random frame number
            rnd = np.random.randint(0, df.shape[0] - 1)
            frame = df.iloc[rnd]
            sample_point = Point(frame[0], frame[1])
            # check if the angles are in the defined range of the ridges
            for seg in ridges:
                rnd_offset = np.random.randint(-0.2 * delta_dist, 0.2 * delta_dist)
                ridge = Segment(seg[0], seg[1])
                if ridge.distance(sample_point) <= delta_dist + rnd_offset:
                    frames[rnd] = frame  # apped frame to our list
                    times.append(time.time() - start)  # save the time spend
                    pbar.update(1)
                    break

    with open("../data/" + str(samples) + "_transition_states_" + datetime.datetime.now().strftime("%Y%m%d%H%M%S"), 'wb') as outfile:
        pickle.dump(frames, outfile, protocol=pickle.HIGHEST_PROTOCOL)

    return frames


def transition_matrix(df, ms_centers, lag_time=1):
    if lag_time == 0:
        print("lag_time is 0!")
        return

    tree = cKDTree(ms_centers)
    points = np.c_[df[0], df[1]]  # fancy zipping
    queries = tree.query(points, n_jobs=2)[1]
    trans_mat = np.zeros(shape=(len(ms_centers), len(ms_centers)))
    for idx in range(df.shape[0] - lag_time):
        old_state = queries[idx]
        new_state = queries[idx + lag_time]
        trans_mat[old_state, new_state] += 1

    # count occurences
    occurences = np.zeros(shape=(len(ms_centers)))
    for label in queries:
        if label == -1:
            continue
        occurences[label] += 1

    # normalization
    # T(A,B)=T(A,B)/N(B)
    for row in range(0, len(ms_centers)):
        for col in range(0, len(ms_centers)):
            t_a_b = trans_mat[row, col]
            t_b = occurences[col]
            trans_mat[row, col] = t_a_b / t_b

    time_spent = occurences / occurences.sum()

    return np.array(trans_mat).T, time_spent


def create_voronoi_ridges(vor):
    line_segments = []
    for simplex in vor.ridge_vertices:
        simplex = np.asarray(simplex)
        if np.all(simplex >= 0):
            line_segments.append([(x, y) for x, y in vor.vertices[simplex]])

    ptp_bound = vor.points.ptp(axis=0)

    center = vor.points.mean(axis=0)
    for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
        simplex = np.asarray(simplex)
        if np.any(simplex < 0):
            i = simplex[simplex >= 0][0]  # finite end Voronoi vertex

            t = vor.points[pointidx[1]] - vor.points[pointidx[0]]  # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[pointidx].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[i] + direction * ptp_bound.max()

            line_segments.append([(vor.vertices[i, 0], vor.vertices[i, 1]),
                                  (far_point[0], far_point[1])])

    return line_segments


def run_micro_simulations(frames, n=10, debug=False):
    sims = {}
    pbar = tqdm(total=len(frames)*n, desc="Simulation microstate transitions")
    for idx in frames:
        runs = []
        for run in range(n):
            # copy topol file to simulations folder
            cmd = 'cp ../data/md_long.top ../simulation/short/topol.top'
            proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            (out, err) = proc.communicate()
            if proc.returncode:
                raise Exception(err)

            # copy gro file to simulations folder
            gro = '../data/frames/' + 'f' + str(idx) + '.gro'
            cmd = 'cp ' + gro + ' ../simulation/short/md.gro'
            proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            (out, err) = proc.communicate()
            if proc.returncode:
                raise Exception(err)

            # run the simulation
            cmd = 'bash --login ../simulation/short/short_mpi.sh'
            proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
            (out, err) = proc.communicate()
            if proc.returncode:
                raise Exception(err)

            # clean up afterwards
            if not debug:
                cmd = 'bash --login ../simulation/short/cleanup.sh'
                proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
                (out, err) = proc.communicate()
                if proc.returncode:
                    raise Exception(err)

            # copy rama file to data folder
            rama = '../data/xvg/' + 'f' + str(idx) + '.rama'
            cmd = 'cp ../simulation/short/rama.xvg ' + rama
            proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            (out, err) = proc.communicate()
            if proc.returncode:
                raise Exception(err)

            # read in the xvg and save the data
            runs.append(read_xvg(rama))
            pbar.update(1)

        # create big array with all the data
        sims[idx] = runs

    pbar.close()

    with open('../data/' + str(len(frames)) + '_micro_sims_' + datetime.datetime.now().strftime("%Y%m%d%H%M%S"), 'wb') as outfile:
        pickle.dump(sims, outfile)

    return sims


def run_macro_simulation(debug=False):
    # run the simulation
    cmd = 'bash --login simulation/long/task_mpi.sh'
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    print("STDOUT:", proc.stdout)
    print("STDERR:", proc.stderr)
    print('task_mpi.sh done')
    (out, err) = proc.communicate()
    if proc.returncode:
       raise Exception(err)
    print('new command')
    # copy gro file of the simulation
    import os

    # Get the current working directory
    current_directory = os.getcwd()

    # Print the current working directory
    print("Current Working Directory:", current_directory)
    cmd = 'cp simulation/long/md.gro data/md_long.gro'
    proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, err) = proc.communicate()
    if proc.returncode:
        raise Exception(err)

    # copy topol file of the simulation
    cmd = 'cp simulation/long/topol.top data/md_long.top'
    proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, err) = proc.communicate()
    if proc.returncode:
        raise Exception(err)

    # copy xtc trajectory file of the simulation
    cmd = 'cp simulation/long/md_corr.xtc data/md_long_corr.xtc'
    proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, err) = proc.communicate()
    if proc.returncode:
        raise Exception(err)

    # save rama file to data folder
    cmd = 'cp simulation/long/rama.xvg data/md_long_nojump_rama.xvg'
    proc = subprocess.Popen(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, err) = proc.communicate()
    if proc.returncode:
        raise Exception(err)

    # clean up afterwards
    if not debug:
        cmd = 'bash --login simulation/long/cleanup.sh'
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
        (out, err) = proc.communicate()
        if proc.returncode:
            raise Exception(err)



if __name__ == '__main__':

    DEBUG = True
    samples = 20

    # first, run a long simulation
    run_macro_simulation(debug=DEBUG)

    df = pd.DataFrame(read_xvg('data/md_long_nojump_rama.xvg'))
    df = apply_offset(df)

    if df.shape[0] < 1000:
        print("Not enough steps saved for analysis.")
        print("Please configure .mdp for GROMACS simulation.")
        exit(1)

    # draw point density
    create_ramachandran_plot(df, "")
    print("Saving figure 'Ramachandran-Plot'")
    plt.savefig('plots/Ramachandran-Plot', dpi=300)
    plt.clf()

    ms_centers = clustering(df)
    vor = Voronoi(ms_centers)

    # save cluster centers to disk
    with open('data/mean_shift_clusters', 'wb') as outfile:
        pickle.dump(ms_centers, outfile, protocol=pickle.HIGHEST_PROTOCOL)

    # draw point density
    create_ramachandran_plot(df, "")
    # draw cluster centers
    plt.scatter(ms_centers.T[0], ms_centers.T[1], linewidths=6, color='black')
    print("Saving figure 'Ramachandran-Meanshift-Plot'")
    plt.savefig('plots/Ramachandran-Meanshift-Plot', dpi=300)
    plt.clf()

    create_ramachandran_plot(df, "")
    # draw cluster centers
    plt.scatter(ms_centers.T[0], ms_centers.T[1], linewidths=6, color='black')

    ridge_lines = create_voronoi_ridges(vor)

    # draw voronoi border
    plot_voronoi_ridges(ridge_lines)
    print("Saving figure 'Ramachandran-Voronoi-Plot'")
    plt.savefig('plots/Ramachandran-Voronoi-Plot', dpi=300)
    plt.clf()

    # load sampled points from disk or sample new points
    sample_files = glob.glob("data/" + str(samples) + "_transition_states_*")
    if len(sample_files):
        with open(sample_files[0], 'rb') as outfile:
            sampled_frames = pickle.load(outfile)
    else:
        # choose only interesting ridges to sample from
        # sampled_frames = sample_boundaries(df, ridge_lines[3:6:1], dist=15, samples=samples)
        # sample from all borders
        sampled_frames = sample_boundaries(df, ridge_lines, dist=15, samples=samples)

    # draw point density
    create_ramachandran_plot(df, "")
    # draw voronoi border
    plot_voronoi_ridges(ridge_lines)
    # draw samples
    plot_sampled_frames(sampled_frames)

    print("Saving figure 'Ramachandran-Transitions-Plot'")
    plt.savefig('plots/Ramachandran-Transitions-Plot', dpi=300)
    plt.clf()

    plot_voronoi_ridges(ridge_lines)
    plot_sampled_frames(sampled_frames, linewidth=1)
    plot_frame_transitions(df, sampled_frames, "")

    print("Saving figure 'Ramachandran-Transitions-Paths-Plot'")
    plt.savefig('plots/Ramachandran-Transitions-Paths-Plot', dpi=300)
    plt.clf()

    paths_files = glob.glob("data/" + str(samples) + "_micro_sims_*")
    if len(paths_files):
        with open(paths_files[0], 'rb') as outfile:
            f_all = pickle.load(outfile)
    else:
        # run small simulations
        frame_keys = sampled_frames.keys()
        # Load simulation results
        u = MDAnalysis.Universe('data/md_long.gro', 'data/md_long_corr.xtc')

        # save frame ids as pdb files
        save_frames_as_pdb(sampled_frames, u)
        # run small simulations and save convert to numpy
        f_all = run_micro_simulations(frame_keys, n=20, debug=DEBUG)

    # plot interesting voronoi region
    plot_voronoi_ridges(ridge_lines)

    # plot all micro transition paths
    traj = samples
    keys = list(f_all.keys())
    random.shuffle(keys)
    for idx in tqdm(keys[:traj], desc="Plotting micro transition paths"):
        c = np.random.rand(3,)
        for t in f_all[idx]:
            npt = pd.DataFrame(np.array(t))
            npt = apply_offset(npt)
            # plot big transition
            prev = df.iloc[idx]
            curr = df.iloc[idx + 1]
            plt.plot([prev[0], curr[0]], [prev[1], curr[1]], color='black', linewidth=2)
            plt.plot([prev[0], curr[0]], [prev[1], curr[1]], color=c, linewidth=1)
            # plot micro transition
            plt.scatter(npt[0][0], npt[1][0], color='black', linewidth=2)
            plt.scatter(npt[0][0], npt[1][0], color=c, linewidth=1)
            plt.plot(npt[0], npt[1], color=c, linewidth=1)
            # plot difference
            plt.plot([npt[0][npt.shape[0]-1], curr[0]], [npt[1][npt.shape[0]-1], curr[1]], linestyle='--', color=c)

    # compute transition matrices and save them
    t_mats = {}
    for i in tqdm(range(1, 4), desc="Computing transition matrices"):
        trans_mat, time_spent = transition_matrix(df, ms_centers, lag_time=i)
        t_mats[i] = {'transition': trans_mat, 'time': time_spent}

    # save transition matrices to disk
    with open('data/transition_matrices', 'wb') as outfile:
        pickle.dump(t_mats, outfile, protocol=pickle.HIGHEST_PROTOCOL)

    print("Saving figure 'Micro-Transitions-Paths-Plot'")
    plt.savefig('plots/Micro-Transitions-Paths-Plot', dpi=300)
    plt.clf()

STDOUT: <_io.BufferedReader name=51>
STDERR: <_io.BufferedReader name=53>
task_mpi.sh done


Exception: ignored

In [19]:
!pip install MDAnalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
